In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Google-Location History Analysis

In [2]:
import json
import os
import time
import datetime
file_path = '/Users/ashmi/Desktop/Takeout/Fit/Daily Aggregations/Location_History.json'
with open(file_path, encoding='utf-8') as data_file:
    data = json.loads(data_file.read())

location_data = pd.DataFrame(data['locations'])

In [3]:
from matplotlib.collections import PatchCollection
from IPython.display import Image
import fiona
from shapely.prepared import prep
from descartes import PolygonPatch
from mpl_toolkits.basemap import Basemap
from shapely.geometry import Point, Polygon, MultiPoint, MultiPolygon
import warnings
warnings.filterwarnings('ignore')
# import cartopy.crs as ccrs

In [4]:
location_data.head()

,accuracy,activity,altitude,heading,latitudeE7,longitudeE7,timestampMs,velocity,verticalAccuracy
0,16,"[{'timestampMs': '1536010106375', 'activity': ...",NaN,NaN,226201999,884314579,1536010110470,NaN,NaN
1,16,"[{'timestampMs': '1536010024596', 'activity': ...",NaN,NaN,226201999,884314579,1536010048469,NaN,NaN
2,16,"[{'timestampMs': '1536009924038', 'activity': ...",NaN,NaN,226201999,884314579,1536009858682,NaN,NaN
3,16,"[{'timestampMs': '1536009711432', 'activity': ...",NaN,NaN,226201999,884314579,1536009797687,NaN,NaN
4,16,NaN,NaN,NaN,226201999,884314579,1536009521292,NaN,NaN


In [5]:
del(data) ## Freeing up space

In [6]:
# convert to typical units
location_data['latitudeE7'] = location_data['latitudeE7']/float(1e7) 
location_data['longitudeE7'] = location_data['longitudeE7']/float(1e7)
location_data['timestampMs'] = location_data['timestampMs'].map(lambda x: float(x)/1000) #to seconds
location_data['datetime'] = location_data.timestampMs.map(datetime.datetime.fromtimestamp)

# Rename fields based on the conversions we just did
location_data.rename(columns={'latitudeE7':'latitude', 'longitudeE7':'longitude', 'timestampMs':'timestamp'}, inplace=True)
location_data = location_data[location_data.accuracy < 1000] #Ignore locations with accuracy estimates over 1000m
location_data.reset_index(drop=True, inplace=True)

In [7]:
location_data.head()

,accuracy,activity,altitude,heading,latitude,longitude,timestamp,velocity,verticalAccuracy,datetime
0,16,"[{'timestampMs': '1536010106375', 'activity': ...",NaN,NaN,22.6202,88.431458,1.536010e+09,NaN,NaN,2018-09-04 02:58:30.470
1,16,"[{'timestampMs': '1536010024596', 'activity': ...",NaN,NaN,22.6202,88.431458,1.536010e+09,NaN,NaN,2018-09-04 02:57:28.469
2,16,"[{'timestampMs': '1536009924038', 'activity': ...",NaN,NaN,22.6202,88.431458,1.536010e+09,NaN,NaN,2018-09-04 02:54:18.682
3,16,"[{'timestampMs': '1536009711432', 'activity': ...",NaN,NaN,22.6202,88.431458,1.536010e+09,NaN,NaN,2018-09-04 02:53:17.687
4,16,NaN,NaN,NaN,22.6202,88.431458,1.536010e+09,NaN,NaN,2018-09-04 02:48:41.292


In [8]:
location_data.dtypes

accuracy                     int64
activity                    object
altitude                   float64
heading                    float64
latitude                   float64
longitude                  float64
timestamp                  float64
velocity                   float64
verticalAccuracy           float64
datetime            datetime64[ns]
dtype: object

In [9]:
print("earliest observed date: {}".format(min(location_data["datetime"]).strftime('%m-%d-%Y')))
print("latest observed date: {}".format(max(location_data["datetime"]).strftime('%m-%d-%Y')))

earliest_obs = min(location_data["datetime"]).strftime('%m-%d-%Y')
latest_obs = max(location_data["datetime"]).strftime('%m-%d-%Y')

earliest observed date: 06-11-2013
latest observed date: 09-04-2018


In [10]:
location_data.head()

,accuracy,activity,altitude,heading,latitude,longitude,timestamp,velocity,verticalAccuracy,datetime
0,16,"[{'timestampMs': '1536010106375', 'activity': ...",NaN,NaN,22.6202,88.431458,1.536010e+09,NaN,NaN,2018-09-04 02:58:30.470
1,16,"[{'timestampMs': '1536010024596', 'activity': ...",NaN,NaN,22.6202,88.431458,1.536010e+09,NaN,NaN,2018-09-04 02:57:28.469
2,16,"[{'timestampMs': '1536009924038', 'activity': ...",NaN,NaN,22.6202,88.431458,1.536010e+09,NaN,NaN,2018-09-04 02:54:18.682
3,16,"[{'timestampMs': '1536009711432', 'activity': ...",NaN,NaN,22.6202,88.431458,1.536010e+09,NaN,NaN,2018-09-04 02:53:17.687
4,16,NaN,NaN,NaN,22.6202,88.431458,1.536010e+09,NaN,NaN,2018-09-04 02:48:41.292


In [11]:
locations_ll = location_data[['latitude', 'longitude']]
locationlist = locations_ll.values.tolist()

In [12]:
import folium

In [13]:
map = folium.Map(location=[49.2, 7.00], zoom_start=12)
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=location_data['datetime'][point]).add_to(map)
map

AttributeError: 'Timestamp' object has no attribute 'get_name'

In [ ]:
degrees_to_radians = np.pi/180.0 
location_data['phi'] = (90.0 - location_data.latitude) * degrees_to_radians 
location_data['theta'] = location_data.longitude * degrees_to_radians
# Compute distance between two GPS points on a unit sphere
location_data['distance'] = np.arccos(
    np.sin(location_data.phi)*np.sin(location_data.phi.shift(-1)) * np.cos(location_data.theta - location_data.theta.shift(-1)) + 
    np.cos(location_data.phi)*np.cos(location_data.phi.shift(-1))) * 6378.100 # radius of earth in km

In [ ]:
location_data['speed'] = location_data.distance/(location_data.timestamp - location_data.timestamp.shift(-1))*3600 #km/hr

In [ ]:
flight_data = pd.DataFrame(data={'end_lat':location_data.latitude,
                             'end_lon':location_data.longitude,
                             'end_datetime':location_data.datetime,
                             'distance':location_data.distance,
                             'speed':location_data.speed,
                             'start_lat':location_data.shift(-1).latitude,
                             'start_lon':location_data.shift(-1).longitude,
                             'start_datetime':location_data.shift(-1).datetime,
                             }).reset_index(drop=True)

In [ ]:
flight_data.head()

In [ ]:
def distance_on_unit_sphere(lat1, long1, lat2, long2):
    # http://www.johndcook.com/python_longitude_latitude.html
    # Convert latitude and longitude to spherical coordinates in radians.
    degrees_to_radians = np.pi/180.0  
    # phi = 90 - latitude
    phi1 = (90.0 - lat1)*degrees_to_radians
    phi2 = (90.0 - lat2)*degrees_to_radians
    # theta = longitude
    theta1 = long1*degrees_to_radians
    theta2 = long2*degrees_to_radians

    cos = (np.sin(phi1)*np.sin(phi2)*np.cos(theta1 - theta2) + 
           np.cos(phi1)*np.cos(phi2))
    arc = np.arccos( cos )
    # Remember to multiply arc by the radius of the earth 
    # in your favorite set of units to get length.
    return arc

In [ ]:
flights = flight_data[(flight_data.speed > 40) & (flight_data.distance > 80)].reset_index()

# Combine instances of flight that are directly adjacent 
# Find the indices of flights that are directly adjacent
_f = flights[flights['index'].diff() == 1]
adjacent_flight_groups = np.split(_f, (_f['index'].diff() > 1).nonzero()[0])

# Now iterate through the groups of adjacent flights and merge their data into
# one flight entry
for flight_group in adjacent_flight_groups:
    idx = flight_group.index[0] - 1 #the index of flight termination
    flights.loc[idx, ['start_lat', 'start_lon', 'start_datetime']] = [flight_group.iloc[-1].start_lat, 
                                                         flight_group.iloc[-1].start_lon, 
                                                         flight_group.iloc[-1].start_datetime]
    # Recompute total distance of flight
    flights.loc[idx, 'distance'] = distance_on_unit_sphere(flights.loc[idx].start_lat,
                                                           flights.loc[idx].start_lon,
                                                           flights.loc[idx].end_lat,
                                                           flights.loc[idx].end_lon)*6378.1   

# Now remove the "flight" entries we don't need anymore.
flights = flights.drop(_f.index).reset_index(drop=True)

# Finally, we can be confident that we've removed instances of flights broken up by
# GPS data points during flight. We can now be more liberal in our constraints for what
# constitutes flight. Let's remove any instances below 200km as a final measure.
flights = flights[flights.distance > 200].reset_index(drop=True)

In [ ]:
fig = plt.figure(figsize=(18,12))

# Plotting across the international dateline is tough. One option is to break up flights
# by hemisphere. Otherwise, you'd need to plot using a different projection like 'robin'
# and potentially center on the Int'l Dateline (lon_0=-180)
# flights = flights[(flights.start_lon < 0) & (flights.end_lon < 0)]# Western Hemisphere Flights
# flights = flights[(flights.start_lon > 0) & (flights.end_lon > 0)] # Eastern Hemisphere Flights

xbuf = 0.2
ybuf = 0.35
min_lat = np.min([flights.end_lat.min(), flights.start_lat.min()])
min_lon = np.min([flights.end_lon.min(), flights.start_lon.min()])
max_lat = np.max([flights.end_lat.max(), flights.start_lat.max()])
max_lon = np.max([flights.end_lon.max(), flights.start_lon.max()])
width = max_lon - min_lon
height = max_lat - min_lat



In [ ]:
import rasterio
from gcmap import GCMapper, Gradient

In [ ]:
# m = Basemap(llcrnrlon=min_lon - width* xbuf,
#             llcrnrlat=min_lat - height*ybuf,
#             urcrnrlon=max_lon + width* xbuf,
#             urcrnrlat=max_lat + height*ybuf,
#             projection='merc',
#             resolution='l',
#             lat_0=min_lat + height/2,
#             lon_0=min_lon + width/2,)




In [ ]:
# m.drawmapboundary(fill_color='#EBF4FA')
# m.drawcoastlines()
# m.drawstates()
# m.drawcountries()
# m.fillcontinents()

# current_date = time.strftime("printed: %a, %d %b %Y", time.localtime())

# for idx, f in flights.iterrows():
#     m.drawgreatcircle(f.start_lon, f.start_lat, f.end_lon, f.end_lat, linewidth=3, alpha=0.4, color='b' )
#     m.plot(*m(f.start_lon, f.start_lat), color='g', alpha=0.8, marker='o')
#     m.plot(*m(f.end_lon, f.end_lat), color='r', alpha=0.5, marker='o' )

# fig.text(0.125, 0.18, "Data collected from 2013-2017 on Android \nPlotted using Python, Basemap \n%s" % (current_date),
#         ha='left', color='#555555', style='italic')
# fig.text(0.125, 0.15, "kivanpolimis.com", color='#555555', fontsize=16, ha='left')
# plt.savefig('flights.png', dpi=150, frameon=False, transparent=False, bbox_inches='tight', pad_inches=0.2)


In [ ]:
flights.head()

In [ ]:
routes = flights
# create gradient to color the routes according to the number of flights
grad = Gradient(((0, 0, 0, 0), (0.5, 204, 0, 153), (1, 255, 204, 230)))
# initialize GCMapper and set data
gcm = GCMapper(cols=grad, height=2000, width=4000)
gcm.set_data(routes['start_lon'], routes['start_lat'], routes['end_lon'],
             routes['end_lat'])#, routes['nb_flights'])
# run & save
img = gcm.draw()
img.save('flights_map_gcmap.png')